<a href="https://colab.research.google.com/github/krishnathawan/SATYA/blob/main/preprocess_verses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faiss-gpu
!pip install sentence-transformers
!pip install pandas
!pip install numpy
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-brnp6gm3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-brnp6gm3
  Resolved https://github.com/huggingface/transformers.git to commit 15bd3e61f8d3680ca472c9314ad07584d20f7b81
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10450848 sha256=98fb3b4555a4402003eab875160bf893ae5684d445399a95b1a278314bf4ee08
  Stored in directory: /tmp/pip-ephem-wheel-cache-thugvtfe/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:


In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
import torch
import faiss
import numpy as np
import pandas as pd

In [2]:
drive.mount('/content/drive')
# paste model path for embeddings generation (fine tuned in previous code)
model_path = '/content/drive/MyDrive/new_model_directory/content/new_fine_tuned'
!ls /content/drive/MyDrive/new_model_directory/content/new_fine_tuned
model = SentenceTransformer(model_path)

Mounted at /content/drive
1_Pooling			   model.safetensors  sentence_bert_config.json  tokenizer.json
config.json			   modules.json       special_tokens_map.json
config_sentence_transformers.json  README.md	      tokenizer_config.json


In [6]:
# paste csv file (dataset of Bhagwad Geeta)
bhagwat_gita_df = pd.read_csv('/Bhagwad_Gita_Verses_English.csv')

# paste csv file (dataset of Patanjali Yoga Sutra)
yoga_sutras_df = pd.read_csv('/Patanjali_Yoga_Sutras_Verses_English.csv')


print(f"Bhagavad Gita DataFrame shape: {bhagwat_gita_df.shape}")
print(f"Patanjali Yoga Sutras DataFrame shape: {yoga_sutras_df.shape}")

sanskrit_shlokas = list(bhagwat_gita_df['Sanskrit ']) + list(yoga_sutras_df['Sanskrit '])

bg_translations = (
    bhagwat_gita_df['Swami Adidevananda'].tolist()
)

pys_translations = (
    yoga_sutras_df['Translation '].tolist()
)

print(f"Number of Sanskrit Shlokas: {len(sanskrit_shlokas)}")
print(f"Number of Bhagavad Gita Translations: {len(bg_translations)}")
print(f"Number of Patanjali Yoga Sutras Translations: {len(pys_translations)}")



Bhagavad Gita DataFrame shape: (700, 9)
Patanjali Yoga Sutras DataFrame shape: (195, 5)
Number of Sanskrit Shlokas: 895
Number of Bhagavad Gita Translations: 700
Number of Patanjali Yoga Sutras Translations: 195


In [7]:
texts = bg_translations + pys_translations
embeddings = model.encode(texts)
embeddings = np.array(embeddings).astype('float32')
print(f"Shape of embeddings: {embeddings.shape}")
index = faiss.IndexHNSWFlat(embeddings.shape[1], 32)
index.add(embeddings)
print(f"Number of vectors in FAISS index: {index.ntotal}")
faiss.write_index(index, 'fine_tuned_embeddings.faiss')

Compiling the model with `torch.compile` and using a `torch.cpu` device is not supported. Falling back to non-compiled mode.


Shape of embeddings: (895, 768)
Number of vectors in FAISS index: 895
